In [1]:
import os
import json
import numpy as np
import pandas as pd
from utils.eval import *
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
a = 45; b= 140
mode = "uni"
tau = 0.13      # 0.13 --- 10% bpm , 0.05 is 3.5%


jdata1 = evaluation_single(a, b, mode, "anchor_zero", tolerance= tau)    # overall acc1 per body segment
jdata2 = evaluation_single(a, b, mode, "anchor_peak", tolerance= tau)    # overall acc1 per body segment
jdata3 = evaluation_single(a, b, mode, "anchor_energy",tolerance= tau)    # overall acc1 per body segment

In [ ]:
skipl = ['lefthand_xy', 'righthand_xy', 'leftfoot_xy', 'rightfoot_xy',
         'bothhand_x_bothfoot_x', 'lefthand_xy_righthand_xy' , 'leftfoot_xy_rightfoot_xy', 'bothhand_x_bothhand_y', 'bothfoot_x_bothfoot_y', 
         ]

flag1 = False
flag2 = False
flag3 = False

print(5*"*", f'{mode}', 5*"*")
print("zero-velocity")
for key in jdata1["bpm_median"].keys():
        
        if key == "both_hand_x":
                flag1 = True
        
        if flag1 and key not in skipl:
                print(key,"---", jdata1["bpm_median"][key]['accuracy'])

        
    
print("\npeak-velocity")
for key in jdata2["bpm_median"].keys():
        
        if key == "both_hand_x":
                flag2 = True
        if flag2 and key not in skipl:
                print(key,"---", jdata2["bpm_median"][key]['accuracy'])
    
print("\nenergy")
for key in jdata3["bpm_median"].keys():
        
        if key == "both_hand_x":
                flag3 = True
        if flag3 and key not in skipl:
                print(key,"---", jdata3["bpm_median"][key]['accuracy'])    

### Proposed method

In [2]:
a, b = 45, 140
tau = 0.13
output_dir = "./tempo_estimation_output"


accuracy_zero = evaluation_multi_segment("anchor_zero", "uni",a,b, tolerance= tau, output_dir= output_dir)
accuracy_peak= evaluation_multi_segment("anchor_peak", "uni",a,b, tolerance= tau, output_dir= output_dir)
accuracy_energy = evaluation_multi_segment("anchor_energy", "uni",a,b, tolerance= tau, output_dir= output_dir)


In [15]:
acc_zero = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['acc']
acc_peak = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['acc']
acc_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['acc']

print("zero", "---", acc_zero)
print("peak", "---", acc_peak)
print("energy", "---", acc_energy)
# tr = load_pickle("/itf-fi-ml/home/sagardu/aist_tempo_est/tempo_estimation_output/tempo_45_140/multi/anchor_zero/bothhand_y_bothfoot_y_torso_y_uni.pkl")

zero --- 73.37807606263982
peak --- 68.38180462341536
energy --- 72.78150633855331


In [18]:
import numpy as np
import pandas as pd
from collections import Counter
with open("genre_symbols_mapping.json", "r") as file:
    genre_id_to_name = json.load(file)
# =============================================================
#  Load ALL fields separately for each anchor type
# =============================================================
acc_zero = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['acc']
acc_peak = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['acc']
acc_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['acc']

# --- Estimated BPM ---
bpm_zero   = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_bpm']
bpm_peak   = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_bpm']
bpm_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_bpm']

# --- Reference BPM ---
ref_bpm_zero   = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_ref_bpm']
ref_bpm_peak   = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_ref_bpm']
ref_bpm_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_ref_bpm']

# --- Genres ---
genres_zero1   = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_genres']
genres_peak1   = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_genres']
genres_energy1 = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_genres']

genres_zero   = [genre_id_to_name[str(g)] for g in genres_zero1]
genres_peak   = [genre_id_to_name[str(g)] for g in genres_peak1]
genres_energy = [genre_id_to_name[str(g)] for g in genres_energy1]

# --- DTS stability ---
dts_zero   = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['dts']
dts_peak   = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['dts']
dts_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['dts']

# --- Segment names ---
seg_zero   = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_seg_names']
seg_peak   = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_seg_names']
seg_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_seg_names']


# =============================================================
#   Computations
# =============================================================

# --- BPM Errors ---
err_zero   = np.abs(bpm_zero   - ref_bpm_zero)
err_peak   = np.abs(bpm_peak   - ref_bpm_peak)
err_energy = np.abs(bpm_energy - ref_bpm_energy)




In [19]:
import numpy as np
import pandas as pd
from collections import Counter

# =============================================================
#  Create DataFrames for each anchor type
# =============================================================

df_zero = pd.DataFrame({
    "acc": acc_zero,
    "bpm": bpm_zero,
    "ref_bpm": ref_bpm_zero,
    "genre": genres_zero,
    "dts": dts_zero,
    "seg": seg_zero
})
df_zero["err"] = np.abs(df_zero["bpm"] - df_zero["ref_bpm"])

df_peak = pd.DataFrame({
    "acc": acc_peak,
    "bpm": bpm_peak,
    "ref_bpm": ref_bpm_peak,
    "genre": genres_peak,
    "dts": dts_peak,
    "seg": seg_peak
})
df_peak["err"] = np.abs(df_peak["bpm"] - df_peak["ref_bpm"])

df_energy = pd.DataFrame({
    "acc": acc_energy,
    "bpm": bpm_energy,
    "ref_bpm": ref_bpm_energy,
    "genre": genres_energy,
    "dts": dts_energy,
    "seg": seg_energy
})
df_energy["err"] = np.abs(df_energy["bpm"] - df_energy["ref_bpm"])


# =============================================================
#  1. OVERALL ACCURACY
# =============================================================
print("\n===== OVERALL ACCURACY =====")
print(f"Zero   : {df_zero['acc'].mean():.4f}")
print(f"Peak   : {df_peak['acc'].mean():.4f}")
print(f"Energy : {df_energy['acc'].mean():.4f}")


# =============================================================
#  2. BPM ERROR STATISTICS
# =============================================================
print("\n===== BPM ERROR (mean ± std) =====")
print(f"Zero   : {df_zero['err'].mean():.2f} ± {df_zero['err'].std():.2f}")
print(f"Peak   : {df_peak['err'].mean():.2f} ± {df_peak['err'].std():.2f}")
print(f"Energy : {df_energy['err'].mean():.2f} ± {df_energy['err'].std():.2f}")


# =============================================================
#  3. GENRE-WISE ANALYSIS
# =============================================================

genre_summary = pd.DataFrame({
    "zero": df_zero.groupby("genre")["err"].mean(),
    "peak": df_peak.groupby("genre")["err"].mean(),
    "energy": df_energy.groupby("genre")["err"].mean()
})

print("\n===== GENRE-WISE BPM ERROR =====")
print(genre_summary)


# =============================================================
#  4. BEST ANCHOR PER GENRE
# =============================================================
best_anchor = genre_summary.idxmin(axis=1)
print("\n===== BEST ANCHOR PER GENRE =====")
print(best_anchor)


# =============================================================
#  5. SEGMENT DOMINANCE
# =============================================================
print("\n===== SEGMENT DOMINANCE =====")
print("Zero   :", dict(Counter(df_zero["seg"])))
print("Peak   :", dict(Counter(df_peak["seg"])))
print("Energy :", dict(Counter(df_energy["seg"])))



===== OVERALL ACCURACY =====
Zero   : 73.3781
Peak   : 68.3818
Energy : 72.7815

===== BPM ERROR (mean ± std) =====
Zero   : 10.49 ± 20.57
Peak   : 6.21 ± 14.87
Energy : 8.69 ± 18.65

===== GENRE-WISE BPM ERROR =====
                       zero       peak     energy
genre                                            
Ballet Jazz       29.504225  18.739683  25.357746
Break              5.972093   3.191892   6.247059
House              3.214706   1.844118   2.429630
Krump             18.379245   9.377778  12.352212
LA style Hip-hop  10.870492   6.257407   9.926667
Lock              11.301031   6.509302   9.866667
Middle Hip-hop     0.843284   1.126829   0.880882
Pop                5.618947   4.683333   4.442000
Street Jazz       25.086792  16.307143  21.708696
Waack             11.038095   4.773684   9.820000

===== BEST ANCHOR PER GENRE =====
genre
Ballet Jazz           peak
Break                 peak
House                 peak
Krump                 peak
LA style Hip-hop      peak
Lock  

## Evaluation: best of N strategy

In [ ]:
segment_names = [["both_hand_y", "both_foot_y"],
                ["both_hand_y", "both_foot_y", "torso_y"]]

a, b = 45, 140
mode = "uni"

json_data = eval_best_of_n(segment_names, a, b, mode, "anchor_zero")

chosen = json_data['both_hand_y_both_foot_y_torso_y']['hit_selected_bpm']
hit_genres_id = json_data['both_hand_y_both_foot_y_torso_y']['hit_genres']

# hit_idx = json_data['both_hand_y_both_foot_y_torso_y']['hit_idx']
# hit_ref_bpm = json_data['both_hand_y_both_foot_y_torso_y']['hit_ref_bpm']
# df_idx = load_pickle("./tempo_estimation_output/tempo_45_140/multi/anchor_zero/bothhand_y_bothfoot_y_torso_y_uni.pkl")
# gtempo = df_idx["gtempo"].to_numpy() # --- can use hit_idx
# hit_bpm = gtempo[hit_idx]


## Octave Search

In [ ]:
hit_idx = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_idx']
hit_ref_bpm = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_ref_bpm']
hit_segment_names = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_seg_names']

df_idx = load_pickle("./tempo_estimation_output/tempo_45_140/multi/anchor_zero/bothhand_y_bothfoot_y_torso_y_uni.pkl")
gtempo = df_idx["gtempo"].to_numpy() # --- can use hit_idx
hit_bpm = gtempo[hit_idx]

In [ ]:
from collections import Counter

tolerance = 0.1  # ±3.5%
half_mask = np.abs(hit_bpm - 0.5 * hit_ref_bpm) <= (tolerance * hit_ref_bpm)
double_mask = np.abs(hit_bpm - 2.0 * hit_ref_bpm) <= (tolerance * hit_ref_bpm)
ref_mask = np.abs(hit_bpm - hit_ref_bpm) <= (tolerance * hit_ref_bpm)

# Counts
half_count = np.sum(half_mask)
double_count = np.sum(double_mask)
ref_count = np.sum(ref_mask)
total = len(hit_bpm)

print("total hits:", total)
print(f"Half-time matches:  {half_count} ({half_count/total:.2%})")
print(f"Double-time matches:{double_count} ({double_count/total:.2%})")
print(f"Reference matches:  {ref_count} ({ref_count/total:.2%})")

# --- Dominant segment for half-time ---
if half_count > 0:
    half_segments = hit_segment_names[half_mask]
    dominant_segment, count = Counter(half_segments).most_common(1)[0]
    print(f"Dominant segment for half-time matches: {dominant_segment} ({count} occurrences)")
else:
    print("No half-time matches found.")


## Body Part Contribution per Dance Genre

In [ ]:
hit_genres_multi = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_genres']
hit_segment_names = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_seg_names']


# part_counts = find_body_contribution_best_of_n(chosen, hit_genres_id)
part_counts = find_body_contribution_multi(hit_genres_multi, hit_segment_names)


In [ ]:
genre_order = ['House', 'Middle Hip-hop', 'Break','LA style Hip-hop',
				'Krump', 'Pop','Lock', 'Waack', 'Ballet Jazz', 'Street Jazz']

part_counts["genre"] = pd.Categorical(
    part_counts["genre"],
    categories=genre_order,
    ordered=True
    )

plt.figure(figsize=(10, 6), dpi=80)
sns.barplot(
    data=part_counts,
    x="percentage",
    y="genre",
    hue="body_part",
    dodge=True,
    order=genre_order,
)

label_map = {
    "both_hand_y": "Hands (y)",
    "both_foot_y": "Feet (y)",
    "torso_y": "Torso (y)"
}
handles, labels = plt.gca().get_legend_handles_labels()
new_labels = [label_map.get(lbl, lbl) for lbl in labels]

# plt.title("Body Part Contribution per Dance Genre")
plt.xlabel("Percentage (%)")
plt.ylabel("Dance Genre")
plt.legend(handles, new_labels,title="Body Part",  loc="lower right")
plt.tight_layout()
plt.show()

